# M3 month without exog

In [3]:
import numpy as np
import pyramid
import pandas as pd
import math
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO
from sklearn import metrics
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
import pandas as pd
from bokeh.models import Span

In [10]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [4]:
df = pd.read_excel('M3C.xls',sheet_name='M3Month')
df = df[df.Category == 'MICRO       ']
df_results_Theta = pd.read_excel('M3Forecast.xls',sheet_name='THETA',header=None)
df_results_FPro = pd.read_excel('M3Forecast.xls',sheet_name='ForecastPro',header=None)
df_results_FX = pd.read_excel('M3Forecast.xls',sheet_name='ForcX',header=None)
df.head()

,Series,N,NF,Category,Starting Year,Starting Month,1,2,3,4,...,135,136,137,138,139,140,141,142,143,144
0,N1402,68,18,MICRO,1990,1,2640.0,2640.0,2160.0,4200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N1403,68,18,MICRO,1990,1,1680.0,1920.0,120.0,1080.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N1404,68,18,MICRO,1990,1,1140.0,720.0,4860.0,1200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N1405,68,18,MICRO,1990,1,180.0,940.0,2040.0,800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N1406,68,18,MICRO,1990,1,2000.0,1550.0,4450.0,3050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
N_series = 9
N = df['N'][N_series]
NF = df['NF'][N_series]
start_year = df['Starting Year'][N_series]
start_month = df['Starting Month'][N_series]
Id = df['Series'][N_series]

K1 = N-12
K2 = N+6
train_data = np.array(df.iloc[N_series,6:K1]).astype(np.float64)
validate_data = np.array(df.iloc[N_series,K1:K2]).astype(np.float64)

In [6]:
forecast_Theta = np.array(df_results_Theta[df_results_Theta[0]==Id]).reshape(-1)[2:].astype(np.int)
forecast_FPro = np.array(df_results_FPro[df_results_FPro[0]==Id]).reshape(-1)[2:].astype(np.int)
forecast_FX = np.array(df_results_FX[df_results_FX[0]==Id]).reshape(-1)[2:].astype(np.int)
del df_results_FPro
del df_results_FX
del df_results_Theta

In [7]:
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    vline = Span(location=49,dimension='height', line_color='red',line_width=1)
    p.renderers.extend([vline])
    
    return p

def plot_arima_comparison(truth_forecast, forecasts,forecasts_rs,forecast_1,forecast_2,forecast_3,forecast_4, title="Comparison", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    forecasts = pd.Series(forecasts)
    forecasts_rs = pd.Series(forecasts_rs)
    true_forecasts = pd.Series(truth_forecast)
    forecasts_1 = pd.Series(forecast_1)
    forecasts_2 = pd.Series(forecast_2)
    forecasts_3 = pd.Series(forecast_3)
    forecasts_4 = pd.Series(forecast_4)
    
    # set up the plot
    p = figure(title=title, plot_height=600, plot_width=1000, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    l1 = p.line(forecasts.index, forecasts.values, color=c2, line_width = 2)
    l2 = p.line(forecasts_rs.index, forecasts_rs.values, color='coral', line_width = 2)
    l3 = p.line(true_forecasts.index, true_forecasts.values, color=c1,line_width=3)
    l4 = p.line(forecasts_1.index, forecasts_1.values, color='brown')
    l5 = p.line(forecasts_2.index, forecasts_2.values, color='goldenrod')
    l6 = p.line(forecasts_3.index, forecasts_3.values, color='purple')
    #l7 = p.line(forecasts_4.index, forecasts_4.values, color='red', line_width = 2)

    legend = Legend(items=[
    ('Forecasted_my_Stepwise'   , [l1]),
    #('Forecasted_my_Stepwise_force' , [l7]),
    ('Forecasted_my_RS' , [l2]),
    ('Observed' , [l3]),
    ('Forecasted_Theta' , [l4]),
    ('Forecasted_FPro' , [l5]),
    ('Forecasted_FX' , [l6]),], location=(0, -30))
    p.add_layout(legend, 'right')
    return p

Loading BokehJS ...

#### Training (Stepwise "full" auto)

In [8]:
from pyramid.arima import auto_arima

stepwise_fit = auto_arima(train_data, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, max_d=4, max_D=2,max_P = 2, max_Q = 2, trace=False,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   50
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -433.276
Date:                Fri, 06 Jul 2018   AIC                            872.553
Time:                        13:10:32   BIC                            878.289
Sample:                             0   HQIC                           874.737
                                 - 50                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   3185.7515    735.469      4.332      0.000    1744.260    4627.243
ar.L1          0.3100      0.152      2.036      0.042       0.012       0.608
sigma2          2e+06   4.21e+05      4.749      0.000    1.17e+06    2.83e+06
===================================================================================
Ljung-Box (Q):                       44.21   Jarque-Bera (JB):                 0.04
Prob(Q):                              0.30   Prob(JB):                         0.98
Heteroskedasticity (H):               1.48   Skew:                             0.01
Prob(H) (two-sided):                  0.43   Kurtosis:                         2.87
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [11]:
#predict in sample = predicting on train sample
in_sample_preds = stepwise_fit.predict_in_sample()
#predicting on test(validate here)
next_validate = stepwise_fit.predict(n_periods=18)
print ("MAPE on train: ",mean_absolute_percentage_error(train_data, in_sample_preds))

MAPE on train:  31.32897963676376


In [12]:
show(plot_arima(train_data, in_sample_preds, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

#### Training (Stepwise force d = 1)

In [13]:
from pyramid.arima import auto_arima

stepwise_fit_force = auto_arima(train_data, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=1, d=1, max_D=2,max_P = 2, max_Q = 2, trace=False, seasonal = True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit_force.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   50
Model:             SARIMAX(0, 1, 2)x(1, 0, 0, 12)   Log Likelihood                -424.218
Date:                            Fri, 06 Jul 2018   AIC                            858.435
Time:                                    13:11:01   BIC                            867.894
Sample:                                         0   HQIC                           862.024
                                             - 50                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     11.3538     15.036      0.755      0.450     -18.117      40.825
ma.L1         -0.5530      0.927     -0.597      0.551      -2.370       1.264
ma.L2         -0.4415      0.439     -1.005      0.315      -1.302       0.419
ar.S.L12       0.3860      0.182      2.123      0.034       0.030       0.742
sigma2      1.762e+06   1.65e+06      1.066      0.287   -1.48e+06       5e+06
===================================================================================
Ljung-Box (Q):                       32.16   Jarque-Bera (JB):                 0.71
Prob(Q):                              0.81   Prob(JB):                         0.70
Heteroskedasticity (H):               1.09   Skew:                             0.29
Prob(H) (two-sided):                  0.86   Kurtosis:                         2.98
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [14]:
in_sample_preds_force = stepwise_fit_force.predict_in_sample()
next_validate_force = stepwise_fit_force.predict(n_periods=18)
print ("MAPE on train:",mean_absolute_percentage_error(train_data, in_sample_preds_force))

MAPE on train: 30.5750674865628


In [15]:
show(plot_arima(train_data, in_sample_preds_force, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

###  Random search

In [16]:
rs_fit = auto_arima(train_data, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                    start_P=0, n_jobs=-1, max_d=4, max_D=2,max_P = 2, max_Q = 2, trace=False,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=False, random=True, random_state=42,  # we can fit a random search (not exhaustive)
                    n_fits=200)

rs_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   50
Model:             SARIMAX(1, 0, 5)x(1, 0, 1, 12)   Log Likelihood                -426.241
Date:                            Fri, 06 Jul 2018   AIC                            872.482
Time:                                    13:11:06   BIC                            891.602
Sample:                                         0   HQIC                           879.763
                                             - 50                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   2942.2266   7155.254      0.411      0.681   -1.11e+04     1.7e+04
ar.L1         -0.9215      0.357     -2.580      0.010      -1.621      -0.221
ma.L1          1.1062      0.488      2.268      0.023       0.150       2.062
ma.L2         -0.1437      0.581     -0.247      0.805      -1.281       0.994
ma.L3         -0.7190      0.594     -1.210      0.226      -1.884       0.446
ma.L4         -0.7720      0.532     -1.450      0.147      -1.816       0.272
ma.L5         -0.3739      0.297     -1.260      0.208      -0.956       0.208
ar.S.L12       0.6670      0.805      0.828      0.407      -0.911       2.245
ma.S.L12      -0.4079      1.061     -0.385      0.701      -2.487       1.671
sigma2      1.917e+06      6.267   3.06e+05      0.000    1.92e+06    1.92e+06
===================================================================================
Ljung-Box (Q):                       26.98   Jarque-Bera (JB):                 0.56
Prob(Q):                              0.94   Prob(JB):                         0.75
Heteroskedasticity (H):               0.94   Skew:                             0.17
Prob(H) (two-sided):                  0.90   Kurtosis:                         3.39
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 1.02e+22. Standard errors may be unstable.
"""

In [17]:
in_sample_preds_rs = rs_fit.predict_in_sample()
next_validate_rs = rs_fit.predict(n_periods=18)
print ("MAPE on train",mean_absolute_percentage_error(train_data, in_sample_preds_rs))

MAPE on train 24.14470006663784


In [18]:
show(plot_arima(train_data, in_sample_preds_rs, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

In [19]:
show(plot_arima_comparison(validate_data,next_validate,next_validate_rs,
                           forecast_Theta,forecast_FPro,forecast_FX,next_validate_force))

In [20]:
print ('MAPE (Stepwise):', mean_absolute_percentage_error(validate_data,next_validate))
print ('MAPE (Stepwise force):', mean_absolute_percentage_error(validate_data,next_validate_force))
print ('MAPE (RS)      :', mean_absolute_percentage_error(validate_data,next_validate_rs))
print ('MAPE (Theta)   :', mean_absolute_percentage_error(validate_data,forecast_Theta))
print ('MAPE (FPro)    :', mean_absolute_percentage_error(validate_data, forecast_FPro))
print ('MAPE (FX)      :', mean_absolute_percentage_error(validate_data,forecast_FX))

MAPE (Stepwise): 19.365917004002515
MAPE (Stepwise force): 30.478067520119023
MAPE (RS)      : 19.101102943086083
MAPE (Theta)   : 17.616221219670344
MAPE (FPro)    : 21.12357924200018
MAPE (FX)      : 20.037732943190264
